# Graph-Based Model Checking for EMMAA

This notebook explores the ut

In [2]:
import json
import pickle
from collections import defaultdict
import boto3
import pandas as pd
import networkx as nx
from indra.statements import *
from emmaa.util import get_s3_client

In [37]:
pd.set_option('display.max_rows', 10)

Load the latest model manager and test results JSON for the EMMAA Ras Machine model:

In [38]:
s3 = get_s3_client()
# Get the Model Manager containing the assembled INDRA Statements
s3_obj = s3.get_object(Bucket='emmaa', Key='results/rasmachine/latest_model_manager.pkl')
mm = pickle.loads(s3_obj['Body'].read())
# Get the Test results JSON
s3_obj = s3.get_object(Bucket='emmaa', Key='results/rasmachine/results_2019-04-30-17-51-24.json')
res = json.load(s3_obj['Body'])

Get the assembled statements from the Model Manager and build a NetworkX DiGraph based on agent names:

In [39]:
stmts = mm.model.assembled_stmts
len(stmts)

5453

In [40]:
edges = []
provenance = defaultdict(list)
for stmt in stmts:
    if not len(stmt.agent_list()) == 2:
        continue
    subj, obj = stmt.agent_list()
    edges.append((subj.name, obj.name))
    if isinstance(stmt, Complex):
        edges.append((obj.name, subj.name))
    provenance[(subj.name, obj.name)].append(stmt)
g = nx.DiGraph()
g.add_edges_from(edges)    

Get the tests and results from the result JSON and compile a list of (source, target) test pairs along with results obtained from the ModelChecker:

In [41]:
g_tests = []
for result in res[1:]:
    code = result['result_json']['result_code']
    if result['english_path']:
        mc_path_len = len(result['english_path'][0])
    else:
        mc_path_len = 0
    tj = result['test_json']
    test_stmt = stmts_from_json([tj])[0]
    if len(test_stmt.agent_list()) == 2:
        subj, obj = test_stmt.agent_list()
        if subj.name != obj.name:
            g_tests.append((tj['type'], subj.name, obj.name, code, mc_path_len))

Using the directed graph compiled from the model statements we look for shortest paths connecting the pairs of test nodes and build a Pandas DataFrame with the results:

In [51]:
rows = []
for stmt_type, subj, obj, code, mc_path_len in g_tests:
    sp_text = ''
    g_path_len = 0
    if subj not in g:
        g_code = 'SUBJECT_NODE_NOT_FOUND'
    elif obj not in g:
        g_code = 'OBJECT_NODE_NOT_FOUND'
    else:
        g_code = 'PATH_FOUND'
        try:
            sp = nx.shortest_path(g, subj, obj)
            g_path_len = len(sp) - 1
            sp_text = ' -> '.join(sp)
        except nx.NetworkXNoPath:
            g_code = 'NO_PATH'
    rows.append((stmt_type, subj, obj, code, mc_path_len, g_code, g_path_len, sp_text))
df = pd.DataFrame.from_records(rows, columns=['test_stmt_type', 'subj', 'obj', 'mc_code',
                                              'mc_path_len', 'g_code', 'g_path_len', 'shortest_path'])

Let's look at some statistics from the results.
First, the number of tests passed by the Model Checker versus the graph:


In [62]:
mc_path = df[(df.mc_code == 'PATHS_FOUND') | (df.mc_code == 'MAX_PATH_LENGTH_EXCEEDED')]
g_path = df[df.g_code == 'PATH_FOUND']
print("Number of MC tests passed: %d (%.1f%%)" % (len(mc_path), 100*len(mc_path)/len(df)))
print("Number of Graph tests passed: %d (%.1f%%)" % (len(g_path), 100*len(g_path)/len(df)))

Number of MC tests passed: 165 (19.0%)
Number of Graph tests passed: 509 (58.7%)


Next, the cases where both the ModelChecker and the Graph found a path:

In [53]:
both_path = mc_path[mc_path.g_code == 'PATH_FOUND']
print("Number of tests passed by both MC and G: %d" % len(both_path))
both_path

Number of tests passed by both MC and G: 165


,test_stmt_type,subj,obj,mc_code,mc_path_len,g_code,g_path_len,shortest_path
0,Activation,AKT1,MTOR,PATHS_FOUND,1,PATH_FOUND,1,AKT1 -> MTOR
2,Activation,AKT1,RPS6KB2,PATHS_FOUND,2,PATH_FOUND,1,AKT1 -> RPS6KB2
15,Activation,BRAF,ELK1,PATHS_FOUND,2,PATH_FOUND,2,BRAF -> MAPK1 -> ELK1
17,Activation,BRAF,MAPK1,PATHS_FOUND,1,PATH_FOUND,1,BRAF -> MAPK1
18,Activation,BRAF,MAPK3,PATHS_FOUND,1,PATH_FOUND,1,BRAF -> MAPK3
...,...,...,...,...,...,...,...,...
846,Phosphorylation,RPS6KB1,IRS1,PATHS_FOUND,1,PATH_FOUND,1,RPS6KB1 -> IRS1
847,Phosphorylation,SRC,CBL,PATHS_FOUND,2,PATH_FOUND,2,SRC -> EGFR -> CBL
849,Phosphorylation,SRC,EGFR,PATHS_FOUND,2,PATH_FOUND,1,SRC -> EGFR
858,Phosphorylation,STK11,TP53,PATHS_FOUND,3,PATH_FOUND,1,STK11 -> TP53


Does it ever happen that the ModelChecker finds a path where there is none in the graph? Apparently not, which is reassuring because the paths found by the ModelChecker should be a strict subset of those in the directed graph:

In [54]:
mc_path[mc_path.g_code != 'PATH_FOUND']

,test_stmt_type,subj,obj,mc_code,mc_path_len,g_code,g_path_len,shortest_path


As an additional sanity check, we look for cases where the ModelChecker yields a *shorter* path than the shortest path in the graph, which should not happen (and doesn't). The rows in the table below consist only of 

In [55]:
df[(df.mc_code == 'PATHS_FOUND') & (df.mc_path_len < df.g_path_len)]

,test_stmt_type,subj,obj,mc_code,mc_path_len,g_code,g_path_len,shortest_path


Next, how often is it that tests failed by the ModelChecker are passed in the Graph?

In [56]:
g_only_path = df[(df.mc_code != 'PATHS_FOUND') & (df.g_code == 'PATH_FOUND')]
mc_fail = len(df[df.mc_code != 'PATHS_FOUND'])
g_only = len(g_only_path)
ratio = g_only / mc_fail
print("Number of MC tests failed: %d" % mc_fail)
print("Number of tests failed by MC and passed by G: %d" % g_only)
print("Ratio: %.3f" % ratio)
g_only_path


Number of MC tests failed: 750
Number of tests failed by MC and passed by G: 392
Ratio: 0.523


,test_stmt_type,subj,obj,mc_code,mc_path_len,g_code,g_path_len,shortest_path
16,Activation,BRAF,ERK,NO_PATHS_FOUND,0,PATH_FOUND,2,BRAF -> ROS1 -> ERK
22,Activation,CASP8,NFKB1,NO_PATHS_FOUND,0,PATH_FOUND,2,CASP8 -> MAPK1 -> NFKB1
28,Activation,CD28,AKT,OBSERVABLES_NOT_FOUND,0,PATH_FOUND,4,CD28 -> GRB2 -> ROS1 -> MTOR -> AKT
30,Activation,CDK2,E2F,NO_PATHS_FOUND,0,PATH_FOUND,2,CDK2 -> CDK4 -> E2F
32,Activation,CDK4,E2F,NO_PATHS_FOUND,0,PATH_FOUND,1,CDK4 -> E2F
...,...,...,...,...,...,...,...,...
860,Phosphorylation,Sirolimus,EIF4EBP1,SUBJECT_MONOMERS_NOT_FOUND,0,PATH_FOUND,2,Sirolimus -> MTOR -> EIF4EBP1
861,Phosphorylation,Sirolimus,RPS6KB1,SUBJECT_MONOMERS_NOT_FOUND,0,PATH_FOUND,2,Sirolimus -> MTOR -> RPS6KB1
862,Phosphorylation,Sirolimus,RPS6KB1,SUBJECT_MONOMERS_NOT_FOUND,0,PATH_FOUND,2,Sirolimus -> MTOR -> RPS6KB1
865,Phosphorylation,TP53,EIF4EBP1,MAX_PATH_LENGTH_EXCEEDED,0,PATH_FOUND,2,TP53 -> APAF1 -> EIF4EBP1


So approximately half of the tests that fail in the ModelChecker pass in the directed graph. Inspection of the specific cases highlights a few recurring scenarios.

First, **statements involving protein families.** In some cases tests involve protein families rather than specific genes, e.g. "BRAF activates ERK", rather than the specific genes MAPK1 or MAPK3. One possibility is that the detailed PySB model doesn't contain sufficient detailed mechanistic statements involving families to find  a connection. However, the directed graph contains a number of (sometimes indirect) edges that create paths satisfying the tests.

In [57]:
g_only_path[g_only_path.obj == 'ERK']

,test_stmt_type,subj,obj,mc_code,mc_path_len,g_code,g_path_len,shortest_path
16,Activation,BRAF,ERK,NO_PATHS_FOUND,0,PATH_FOUND,2,BRAF -> ROS1 -> ERK
59,Activation,EGFR,ERK,NO_PATHS_FOUND,0,PATH_FOUND,2,EGFR -> ROS1 -> ERK
93,Activation,ERBB2,ERK,NO_PATHS_FOUND,0,PATH_FOUND,3,ERBB2 -> EGFR -> ROS1 -> ERK
115,Activation,FGFR1,ERK,NO_PATHS_FOUND,0,PATH_FOUND,1,FGFR1 -> ERK
131,Activation,GRB2,ERK,NO_PATHS_FOUND,0,PATH_FOUND,2,GRB2 -> ROS1 -> ERK
...,...,...,...,...,...,...,...,...
290,Activation,SOS1,ERK,NO_PATHS_FOUND,0,PATH_FOUND,2,SOS1 -> FGFR1 -> ERK
594,Inhibition,DUSP1,ERK,NO_PATHS_FOUND,0,PATH_FOUND,3,DUSP1 -> EGFR -> ROS1 -> ERK
622,Inhibition,NF1,ERK,NO_PATHS_FOUND,0,PATH_FOUND,3,NF1 -> ALK -> ROS1 -> ERK
631,Inhibition,PTEN,ERK,NO_PATHS_FOUND,0,PATH_FOUND,2,PTEN -> ROS1 -> ERK


Second, **statements types not handled by the Model Checker.** Complex statements, which describe binding between two proteins, are currently not handled by the INDRA Model Checker for technical reasons. These were often trivially satisfied by links in the graph.

In [58]:
g_only_path[g_only_path.mc_code == 'STATEMENT_TYPE_NOT_HANDLED']

,test_stmt_type,subj,obj,mc_code,mc_path_len,g_code,g_path_len,shortest_path
323,Complex,ARHGAP35,RASA1,STATEMENT_TYPE_NOT_HANDLED,0,PATH_FOUND,2,ARHGAP35 -> GRB2 -> RASA1
328,Complex,CBL,EGFR,STATEMENT_TYPE_NOT_HANDLED,0,PATH_FOUND,1,CBL -> EGFR
329,Complex,CCND1,CDK4,STATEMENT_TYPE_NOT_HANDLED,0,PATH_FOUND,1,CCND1 -> CDK4
330,Complex,CCND2,CDK4,STATEMENT_TYPE_NOT_HANDLED,0,PATH_FOUND,1,CCND2 -> CDK4
334,Complex,E2F1,RB1,STATEMENT_TYPE_NOT_HANDLED,0,PATH_FOUND,1,E2F1 -> RB1
...,...,...,...,...,...,...,...,...
375,Complex,PIK3R1,PTPN11,STATEMENT_TYPE_NOT_HANDLED,0,PATH_FOUND,1,PIK3R1 -> PTPN11
376,Complex,PTEN,TP53,STATEMENT_TYPE_NOT_HANDLED,0,PATH_FOUND,1,PTEN -> TP53
377,Complex,RAF1,RB1,STATEMENT_TYPE_NOT_HANDLED,0,PATH_FOUND,1,RAF1 -> RB1
381,Complex,TP73,YAP1,STATEMENT_TYPE_NOT_HANDLED,0,PATH_FOUND,1,TP73 -> YAP1


Third, cases where the **specific state of the subject or object was not found in the model**. These represent cases where the graph is explicitly ignoring details of the subject or object state in determining the existence of a causal link. Determining the proportion of paths produced by the directed graph that are mechanistically defensible will require inspection of the provenance of the underlying statements.

In [59]:
g_only_path[(g_only_path.mc_code == 'SUBJECT_MONOMERS_NOT_FOUND') |
            (g_only_path.mc_code == 'OBSERVABLES_NOT_FOUND')]

,test_stmt_type,subj,obj,mc_code,mc_path_len,g_code,g_path_len,shortest_path
28,Activation,CD28,AKT,OBSERVABLES_NOT_FOUND,0,PATH_FOUND,4,CD28 -> GRB2 -> ROS1 -> MTOR -> AKT
56,Activation,EGFR,AKT,OBSERVABLES_NOT_FOUND,0,PATH_FOUND,2,EGFR -> MTOR -> AKT
68,Activation,EGFR,PLCE1,OBSERVABLES_NOT_FOUND,0,PATH_FOUND,2,EGFR -> BRAF -> PLCE1
76,Activation,EGFR,AKT,OBSERVABLES_NOT_FOUND,0,PATH_FOUND,2,EGFR -> MTOR -> AKT
78,Activation,EGFR,AKT,OBSERVABLES_NOT_FOUND,0,PATH_FOUND,2,EGFR -> MTOR -> AKT
...,...,...,...,...,...,...,...,...
853,Phosphorylation,SRC,PDGFR,OBSERVABLES_NOT_FOUND,0,PATH_FOUND,3,SRC -> EGFR -> ARAF -> PDGFR
859,Phosphorylation,Sirolimus,AKT,SUBJECT_MONOMERS_NOT_FOUND,0,PATH_FOUND,2,Sirolimus -> MTOR -> AKT
860,Phosphorylation,Sirolimus,EIF4EBP1,SUBJECT_MONOMERS_NOT_FOUND,0,PATH_FOUND,2,Sirolimus -> MTOR -> EIF4EBP1
861,Phosphorylation,Sirolimus,RPS6KB1,SUBJECT_MONOMERS_NOT_FOUND,0,PATH_FOUND,2,Sirolimus -> MTOR -> RPS6KB1
